In [ ]:
'''
# 1*1 convolution 减少通道，从而减小计算量 也叫network in network NIN

#GoogleLeNet创建



#Inception块

#part 1
self.branch_pool = nn.Conv2d(in_channels,24,kernel_size=1) #实例化1*1卷积

branch_pool=F.avg_pool2d(x,kernel_size=3,stride=1,padding=1) #先平均池化采样
branch_pool=self.branch_pool(branch_pool) #再1*1卷积

#part 2
self.branch1x1=nn.Conv2d(in_channels,16,kernel_size=1)
branch1x1=self.branch1x1(x)  #只进行outchanne=16的1*1卷积

#part 3
self.branch5x5_1=nn.Conv2d(in_channels,16,kernel_size=1)
self.branch5x5_2=nn.Conv2d(16,24,kernel_size=5,padding=2) #16对应上行output channel
branch5x5=self.branch5x5_1(x)
branch5x5=self.branch5x5_2(branch5x5) 

#part 4
self.branch3x3_1=nn.Conv2d(in_channels,16,kernel_size=1)
self.branch3x3_2=nn.Conv2d(16,24,kernel_size=3,padding=1) #16对应上行output channel
self.branch3x3_3=nn.Conv2d(24,24,kernel_size=3,padding=1) #16对应上行output channel
branch3x3=self.branch3x3_1(x)
branch3x3=self.branch3x3_2(branch3x3) 
branch3x3=self.branch3x3_3(branch3x3) 



#Concatenate 拼接
outputs=[branch1x1,branch3x3,branch5x5,branch_pool]
return torch.cat(outputs,dim=1) #[b,c,w,h]在channel维度进行拼接，dim=1  torch.cat-拼接的缩写
'''

In [ ]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    
        self.branch5x5_1 = nn.Conv2d(in_channels,16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)
    
        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
     
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
    
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, dim=1)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5) #上述注释部分中4个part拼接后channel=24+16+24+24=88
        
        self.incep1 = InceptionA(in_channels=10) #卷积1的out_channel=10，因此incep1的in_channel=10
        
        self.incep2 = InceptionA(in_channels=20) #同上
        
        self.mp = nn.MaxPool2d(2)
        
        self.fc = nn.Linear(1408, 10) #线性化，outchannel=10因为只有十个数字
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x))) #卷积-maxpool-relu
        x = self.incep1(x)                 #第一次incep,inputchannel变为88
        x = F.relu(self.mp(self.conv2(x))) #卷积-maxpool-relu
        x = self.incep2(x)                #第一次incep,inputchannel变为20
        x = x.view(in_size, -1)
        x = self.fc(x)
